<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/FaceFusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Silent Installation**
import os

# Clone repository quietly
!git clone -q https://github.com/splendormagic/Bahaa-Ai-DeepFake roop > /dev/null 2>&1
%cd roop

# Modify requirements.txt silently
!sed -i 's/numpy==1.24.3/numpy==1.26.4/g' requirements.txt > /dev/null 2>&1
!sed -i 's/tensorflow==2.14.0/tensorflow==2.16.1/g' requirements.txt > /dev/null 2>&1

# Install dependencies quietly
!pip install --quiet --no-warn-conflicts --extra-index-url https://download.pytorch.org/whl/cu118 -r requirements.txt --force-reinstall > /dev/null 2>&1

# Set CUDA environment variables
os.environ["CUDA_PATH"] = "/usr/local/cuda"
os.environ["PATH"] += ":/usr/local/cuda/bin"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64"

print("Installation done")

In [ ]:
#@title **Run Job (Google Drive Integration with Result Display)**
import os
from google.colab import drive
from IPython.display import display, Image

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Configure paths
DRIVE_DIR = "/content/drive/MyDrive/Deepfake"
INPUT_DIR = os.path.join(DRIVE_DIR, "input")
OUTPUT_DIR = os.path.join(DRIVE_DIR, "output")
SOURCE_IMG = os.path.join(INPUT_DIR, "source.png")
TARGET_IMG = os.path.join(INPUT_DIR, "target.png")
RESULT_IMG = os.path.join(OUTPUT_DIR, "result.png")

# Create directories
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Validate input files
if not (os.path.exists(SOURCE_IMG) and os.path.exists(TARGET_IMG)):
    raise FileNotFoundError(f"Missing source.png or target.png in {INPUT_DIR}")

# Run processing
%cd /content/roop
!python run.py --source {SOURCE_IMG} --target {TARGET_IMG} --output {RESULT_IMG} --execution-provider cuda --frame-processor face_swapper

# Display result
print("Deepfake Result:")
display(Image(RESULT_IMG))
print(f"Result saved to: {RESULT_IMG}")

In [ ]:
#@title **Run Job (Local Files)**
import os
from google.colab import files
from IPython.display import display, Image

# Navigate to project directory
%cd /content/roop

# Check if files exist, upload if missing
if not (os.path.exists("source.png") and os.path.exists("target.png")):
    print("Uploading source and target images...")
    uploaded = files.upload()
    if 'source.png' not in uploaded or 'target.png' not in uploaded:
        raise FileNotFoundError("Please upload both source.png and target.png")

# Run processing
!python run.py --source source.png --target target.png --output result.png --execution-provider cuda --frame-processor face_swapper

# Display result
print("Deepfake Result:")
display(Image("result.png"))
print("Result saved to: /content/roop/result.png")